In [ ]:
MARKET = 'NSE'

In [ ]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [ ]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format # set float precision with comma
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [ ]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [ ]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [ ]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

# The core engine with pre and post processors

In [ ]:
# ** EXECUTION
# * Core engine that processes functions and delivers results

# .preprocessing data for the core engine


def pre_process(cts):
    """Generates tuples for input to the engine"""

    try:
        symbol = cts.symbol
        output = ((cts, None),)

    except AttributeError as ae1:  # it's an iterable!
        try:
            symbols = [c.symbol for c in cts]

            if len(symbols) == 1:
                output = ((cts[0], None),)
            else:
                output = ((c, None) for c in cts)

        except AttributeError as ae2:  # 2nd value is MarketOrder!
            try:
                output = tuple(cts)
            except:
                print(f"Unknown error in {ae2}")
                output = None

    return tuple(output)


# .make name for symbol being processed by the engine
def make_name(cts):
    """Generates name for contract(s)"""
    try:
        output = [
            c.symbol
            + c.lastTradeDateOrContractMonth[-4:]
            + c.right
            + str(c.strike)
            + ".."
            for c in cts
        ]

    except TypeError as te:  # single non-iterable element
        if cts != "":  # not empty!
            output = (
                cts.symbol
                + cts.lastTradeDateOrContractMonth[-4:]
                + cts.right
                + str(cts.strike)
            )
        else:
            output = cts

    except AttributeError as ae1:  # multiple (p, s) combination
        try:
            output = [
                c[0].symbol
                + c[0].lastTradeDateOrContractMonth[-4:]
                + c[0].right
                + str(c[0].strike)
                + ".."
                for c in cts
            ]
        except TypeError as te2:
            output = (
                cts[0].symbol
                + cts[0].lastTradeDateOrContractMonth[-4:]
                + cts[0].right
                + str(cts[0].strike)
            )

    return output


# .the core engine
async def executeAsync(
    ib: IB(),
    algo: Callable[..., Coroutine],  # coro name
    cts: Union[Contract, pd.Series, list, tuple],  # list of contracts
    CONCURRENT: int = 44,  # to prevent overflows put 44 * (TIMEOUT-1)
    TIMEOUT: None = None,  # if None, no progress messages shown
    post_process: Callable[
        [set, pathlib.Path, str], pd.DataFrame
    ] = None,  # If checkpoint is needed
    DATAPATH: pathlib.Path = None,  # Necessary for post_process
    OP_FILENAME: str = "",  # output file name
    SHOW_TQDM: bool = True,  # Show tqdm bar instead of individual messages
    REUSE: bool = False,  # Reuse the OP_FILENAME supplied
    **kwargs,
) -> pd.DataFrame:

    tasks = set()
    results = set()
    remaining = pre_process(cts)
    last_len_tasks = 0  # tracking last length for error catch

    # Set pbar
    if SHOW_TQDM:
        pbar = tqdm(
            total=len(remaining),
            desc=f"{algo.__name__}: ",
            bar_format=BAR_FORMAT,
            ncols=80,
            leave=False,
        )

    # Get the results
    while len(remaining):

        # Tasks limited by concurrency
        if len(remaining) <= CONCURRENT:

            tasks.update(
                asyncio.create_task(algo(ib, c, **kwargs), name=make_name(c))
                for c in remaining
            )

        else:

            tasks.update(
                asyncio.create_task(algo(ib, c, **kwargs), name=make_name(c))
                for c in remaining[:CONCURRENT]
            )

        # Execute tasks
        while len(tasks):

            done, tasks = await asyncio.wait(
                tasks, timeout=TIMEOUT, return_when=asyncio.ALL_COMPLETED
            )

            # Remove dones from remaining
            done_names = [d.get_name() for d in done]
            remaining = [c for c in remaining if make_name(c) not in done_names]

            # Update results and checkpoint
            results.update(done)

            # Checkpoint the results
            if post_process:

                output = post_process(
                    results=results,
                    DATAPATH=DATAPATH,
                    REUSE=REUSE,
                    LAST_RUN=False,
                    OP_FILENAME=OP_FILENAME,
                )

                if not output.empty:
                    REUSE = False  # for second run onwards

            else:
                output = results

            if TIMEOUT:

                if remaining:

                    if SHOW_TQDM:
                        pbar.update(len(done))

                    else:
                        print(
                            f"\nDone {algo.__name__} for {done_names[:2]} {len(results)} out of {len(cts)}. Pending {[make_name(c) for c in remaining][:2]}"
                        )

                # something wrong. Task is not progressing
                if (len(tasks) == last_len_tasks) & (len(tasks) > 0):
                    print(
                        f"\n @ ALERT @: Tasks are failing !\n"+ \
                        f"Pending {len(tasks)} " + \
                        f"tasks such as {[t.get_name() for t in tasks][:3]}\n" +\
                        f"... will be killed in 5 seconds\n"
                    )
                    dn, pend = await asyncio.wait(tasks, timeout=5.0)
                    if len(dn) > 0:
                        results.update(dn)

                    tasks.difference_update(dn)
                    tasks.difference_update(pend)

                    pend_names = [p.get_name() for p in pend]
                    # remove pending from remaining
                    remaining = [c for c in remaining if make_name(c) not in pend_names]

                # re-initialize last length of tasks
                last_len_tasks = len(tasks)

    # Make the final output, based on REUSE status

    if OP_FILENAME:
        df = post_process(
            results=set(),  # Empty dataset
            DATAPATH=DATAPATH,
            REUSE=REUSE,
            LAST_RUN=True,
            OP_FILENAME=OP_FILENAME,
        )
    else:
        df = output

    if SHOW_TQDM:

        pbar.update(len(done))
        pbar.refresh()
        pbar.close()

    return df


# .Process output into dataframes
def post_df(
    results: set,
    DATAPATH: pathlib.Path,
    REUSE: bool,
    LAST_RUN: bool,
    OP_FILENAME: str = "",
) -> pd.DataFrame():

    if results:
        df = pd.concat([r.result() for r in results if r], ignore_index=True)

        if OP_FILENAME:

            if REUSE:

                # load the existing file

                try:
                    df_old = pd.read_pickle(DATAPATH.joinpath(OP_FILENAME))

                    # Save old temporarily
                    df_old.to_pickle(DATAPATH.joinpath("z_temp_" + OP_FILENAME))

                except FileNotFoundError:
                    pass

            df.to_pickle(DATAPATH.joinpath(OP_FILENAME))

    else:

        if LAST_RUN:  # Merge new and old df (if available)

            if OP_FILENAME:

                try:
                    df_old = pd.read_pickle(DATAPATH.joinpath("z_temp_" + OP_FILENAME))
                    # cleanup temp file
                    os.remove(DATAPATH.joinpath("z_temp_" + OP_FILENAME))

                except FileNotFoundError:
                    df_old = pd.DataFrame([])

                df_new = pd.read_pickle(DATAPATH.joinpath(OP_FILENAME))
                df = df_new.append(df_old).reset_index(drop=True)
                df.to_pickle(DATAPATH.joinpath(OP_FILENAME))

        else:
            df = pd.DataFrame([])  # results are not yet ready!

    return df

# Quick price algo

In [ ]:
async def quick_price(ib: IB, contract: Contract) -> pd.DataFrame:
    
    if isinstance(contract, tuple):
        contract = contract[0]

    result = defaultdict(dict)

    ticks = ib.reqHistoricalTicks(
            contract=contract,
            startDateTime="",
            endDateTime=datetime.datetime.now(),
            numberOfTicks=1,
            whatToShow="Bid_Ask",
            useRth=False,
            ignoreSize=False,
        )
    
    await asyncio.sleep(1) # to make this asyncio friendly!

    # extract bid and ask price
    try:
        bid_ask = ticks[-1]  # bid ask is not availble for Index securities!
        result["bid"] = bid_ask.priceBid
        result["ask"] = bid_ask.priceAsk

    except IndexError:
        print(
            f"\nNo bid-ask for {contract.localSymbol} of secType: {contract.secType}\n"
        )
        result["bid"] = np.nan
        result["ask"] = np.nan
        
    ticks = ib.reqHistoricalTicks(
            contract=contract,
            startDateTime="",
            endDateTime=datetime.datetime.now(),
            numberOfTicks=1,
            whatToShow="Trades",
            useRth=False,
            ignoreSize=False,
        )
    
    await asyncio.sleep(1) # to make this asyncio friendly!

    # extract last reported price
    try:
        # pick reported price if available
        result["last"] = [t.price for t in ticks if not t.tickAttribLast.unreported][
            -1
        ]
    except IndexError:
        # pick up last tick price
        try:
            result["last"] = ticks[-1].price
        except IndexError:
            result["last"] = np.nan

    # . build the df
    df_pr = pd.DataFrame(
        [
            pd.Series(contract.conId, name="conId"),
            pd.Series(contract.symbol, name="symbol"),
            pd.Series(contract.localSymbol, name="localSymbol"),
            pd.Series(result["bid"], name="bid", dtype="float64"),
            pd.Series(result["ask"], name="ask", dtype="float64"),
            pd.Series(result["last"], name="last", dtype="float64"),
        ]
    ).T

    # . use bid-ask avg if last price is not available
    df_pr = df_pr.assign(
        price=df_pr["last"].combine_first(df_pr[["bid", "ask"]].mean(axis=1))
    )

    return df_pr

In [ ]:
async def quick_price_async(ib: IB, contract: Contract) -> pd.DataFrame:
    
    if isinstance(contract, tuple):
        contract = contract[0]
    
    result = defaultdict(dict)
    
    ticks = await asyncio.gather(ib.reqHistoricalTicksAsync(
                                    contract=contract,
                                    startDateTime='',
                                    endDateTime=datetime.datetime.now(),
                                    numberOfTicks=1,
                                    whatToShow='Bid_Ask',
                                    useRth=False,
                                    ignoreSize=False),
                                ib.reqHistoricalTicksAsync(
                                    contract=contract,
                                    startDateTime='',
                                    endDateTime=datetime.datetime.now(),
                                    numberOfTicks=1,
                                    whatToShow='Trades',
                                    useRth=False,
                                    ignoreSize=False))
    
    # extract bid and ask price
    try:
        bid_ask = ticks[0][-1] # bid ask is not availble for Index securities!
        result['bid'] = bid_ask.priceBid
        result['ask'] = bid_ask.priceAsk
        
    except IndexError:
        print(f'\nNo bid-ask for {contract.localSymbol} of secType: {contract.secType}')
        result['bid'] = np.nan
        result['ask'] = np.nan       

    # extract last reported price
    try:
        # pick reported price if available
        result['last'] = [t.price for t in ticks[1] 
                      if not t.tickAttribLast.unreported][-1]
    except IndexError:
        # pick up last tick price
        
        try:
            result['last'] = ticks[1][-1].price
        except IndexError:
            result['last'] = np.nan
            
    
    # . build the df
    df_pr = pd.DataFrame([pd.Series(contract.conId, name='conId'),
                          pd.Series(contract.symbol, name='symbol'),
                          pd.Series(contract.localSymbol, name='localSymbol'),
                          pd.Series(result['bid'], name='bid', dtype='float64'), 
                          pd.Series(result['ask'], name='ask', dtype='float64'), 
                          pd.Series(result['last'], name='last', dtype='float64')]).T

    # . use bid-ask avg if last price is not available
    df_pr = df_pr.assign(price=df_pr['last'].combine_first(df_pr[['bid', 'ask']].mean(axis=1)))

    return df_pr

In [ ]:
# * IMPORTS
import asyncio
import datetime
import math
import os
import pathlib
from io import StringIO
from typing import Callable, Coroutine, Union

import IPython as ipy
import numpy as np
import pandas as pd
import requests
from ib_insync import IB, Contract, MarketOrder, Option, util
from tqdm import tqdm

from support import Timer, Vars, calcsdmult_df, get_dte, get_prob, quick_pf, yes_or_no

from collections import defaultdict

In [ ]:
# * FUNCTION INPUTS
cts = df_unds.contract.unique().tolist() # !!! DATA LIMITER

In [ ]:
# ** SETUP
THIS_FOLDER = ""
BAR_FORMAT = "{desc:<10}{percentage:3.0f}%|{bar:25}{r_bar}{bar:-10b}"

In [ ]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    ib.client.setConnectOptions('PACEAPI')
    df = ib.run(executeAsync(
        ib = ib,
        algo = quick_price,
        cts = cts,
        CONCURRENT = 40,
        TIMEOUT = 5,
        SHOW_TQDM = False,
    ))

In [4]:
import pandas as pd
df1 = pd.concat([pd.Series([1, 2, 3], name='numbers'), 
                 pd.Series(['a', 'b', 'c'], name='letters')], axis=1)
df2 = pd.concat([pd.Series([4, 6, 5], name='numbers'), 
                 pd.Series(['x', 'y', 'z'], name='letters')], axis=1)

In [26]:
results = []

In [27]:
results = results+tuple([df1])

TypeError: can only concatenate list (not "tuple") to list

In [20]:
results = results + tuple([df2])

In [25]:
pd.concat(results)

,numbers,letters
0,1,a
1,2,b
2,3,c
0,4,x
1,6,y
2,5,z


In [45]:
class Foo(object):
  def __init__(self, val=2):
     self.val = val
  def __getstate__(self):
     print("I'm being pickled")
     self.val *= 2
     return self.__dict__
  def __setstate__(self, d):
     print("I'm being unpickled with these values: " + repr(d))
     self.__dict__ = d
     self.val *= 3

import pickle
f = Foo()
f_data = pickle.dumps(f)
f_new = pickle.loads(f_data)

I'm being pickled
I'm being unpickled with these values: {'val': 4}


In [35]:
d.tzinfo.utcoffset(d)

AttributeError: 'NoneType' object has no attribute 'utcoffset'